## Element-wise dot product

### Numpy

In [46]:
# %load test_numpy
from numpy.core.umath_tests import inner1d
import numpy as np
n=1000
x = np.random.normal(loc = 0.0, scale = 1.0, size = 2*n) 
y = np.random.normal(loc = 0.0, scale = 1.0, size = 2*n) 
z = np.random.normal(loc = 0.0, scale = 1.0, size = 2*n) 
S = np.concatenate(([x],[y],[z]), axis=0).T
S1 = S[:n,:]
S2 = S[n:,:]

In [47]:
%%timeit

np.sum(S1*S2, axis=1)

9.24 µs ± 9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [48]:
%%timeit

inner1d(S1,S2)

2.93 µs ± 2.24 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [49]:
%%timeit

np.cos(x)

20.9 µs ± 61.4 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [50]:
%%timeit

[np.dot(s1, s2) for s1, s2 in zip(S1,S2)]

502 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Torch

In [51]:
# %load test_torch
import torch
n=1000
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dist = torch.distributions.normal.Normal(loc = 0.0, scale = 1.0)
x = dist.sample((2 * n,1)).to(device)
y = dist.sample((2 * n,1)).to(device)
z = dist.sample((2 * n,1)).to(device)
S = torch.cat((x,y,z), axis=1).to(device)
S1 = S[:n,:].to(device)
S2 = S[n:,:].to(device)
print(f"Using torch device: {device}")


Using torch device: cuda


In [52]:
%%timeit

torch.sum(S1*S2, axis=1)

14.8 µs ± 45.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [53]:
%%timeit

torch.cos(x)

5.75 µs ± 26.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Conclusion

Fastest element-wise dot product method for problem: numpy.core.umath_tests import inner1d

## Trapezoidal Integration

### Numpy

In [54]:
# %load test_numpy
import numpy as np
n=1000
x = np.random.normal(loc = 0.0, scale = 1.0, size = 2*n) 
y = np.random.normal(loc = 0.0, scale = 1.0, size = 2*n) 
z = np.random.normal(loc = 0.0, scale = 1.0, size = 2*n) 
S = np.concatenate(([x],[y],[z]), axis=0).T
S1 = S[:n,:]
S2 = S[n:,:]

In [55]:
%%timeit

np.trapz(x)

9.61 µs ± 28.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Torch

In [56]:
# %load test_torch
import torch
n=1000
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dist = torch.distributions.normal.Normal(loc = 0.0, scale = 1.0)
x = dist.sample((2 * n,1)).to(device)
y = dist.sample((2 * n,1)).to(device)
z = dist.sample((2 * n,1)).to(device)
S = torch.cat((x,y,z), axis=1).to(device)
S1 = S[:n,:].to(device)
S2 = S[n:,:].to(device)
print(f"Using torch device: {device}")


Using torch device: cuda


In [57]:
%%timeit

torch.trapz(x)

40.1 µs ± 59.4 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Conclusion

Numpy

## Redundant sampling points

### Numpy

In [69]:
# %load test_numpy
import numpy as np
n=1000
x = np.random.normal(loc = 0.0, scale = 1.0, size = 2*n) 
y = np.random.normal(loc = 0.0, scale = 1.0, size = 2*n) 
z = np.random.normal(loc = 0.0, scale = 1.0, size = 2*n) 
S = np.concatenate(([x],[y],[z]), axis=0).T
S1 = S[:n,:]
S2 = S[n:,:]
print(S1.shape)

(1000, 3)


In [68]:
test_cond = np.concatenate((np.ones(n),np.zeros(n)))

In [71]:
%%timeit

np.where(test_cond, inner1d(S,S), 0)

8.52 µs ± 6.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [72]:
%%timeit

inner1d(S,S) * test_cond

6.15 µs ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Conclusion

Using np.where does not optimise over element-wise vector multiplication with the boolean condition.

## Spacecraft pointing history

In [128]:
# %load test_numpy.py
import numpy as np
n = 1000
t = np.linspace(0,2 * np.pi, n)

def rot(theta):
    return np.array([
        [+np.cos(theta), -np.sin(theta)],
        [+np.sin(theta), +np.cos(theta)]
    ])


def sin_a_plus_b(cos_a, sin_a, b):
    return sin_a * np.cos(b) + np.sin(b) * cos_a

def cos_a_plus_b(cos_a, sin_a, b):
    return cos_a * np.cos(b) - sin_a * np.sin(b)

def unit_vector_trajectory(cos_a, sin_a, b):
    return np.array([
        cos_a_plus_b(cos_a, sin_a, b),
        sin_a_plus_b(cos_a, sin_a, b)]).T
    
bod_t = np.pi / 4
bod_v = np.array([1,1])/np.sqrt(2)
c_att = np.cos(t)
s_att = np.sin(t)

In [118]:
%%timeit

c_att = np.cos(t)
s_att = np.sin(t)

24.6 µs ± 86.3 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [119]:
%%timeit

np.matmul(rot(t).transpose([2,0,1]) , bod_v)

68.5 µs ± 54.6 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [129]:
%%timeit

unit_vector_trajectory(np.cos(t), np.sin(t), bod_t)

36.9 µs ± 84.4 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [130]:
%%timeit

unit_vector_trajectory(c_att, s_att, bod_t)

11.7 µs ± 29.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [139]:
res1 = np.matmul(rot(t).transpose([2,0,1]), bod_v)
res2 = unit_vector_trajectory(np.cos(t), np.sin(t), bod_t)

np.sum(np.abs(res1-res2))
# assert np.all(res1 == res2)

1.4299672557172016e-13

In [143]:
time_array_1 = np.linspace(0,np.pi, 10 )
time_array_2 = np.linspace(0,np.pi,1000)

In [173]:
omega = np.sin(time_array_1)
from scipy import integrate, interpolate
init_theta = 0

In [179]:
%%timeit
omega2 = interpolate.interp1d(time_array_1, omega)(time_array_2)
theta =  integrate.cumtrapz(omega2, time_array_2, initial=0) + init_theta
alpha = np.diff(omega2)
alpha = np.concatenate((alpha, [alpha[-1]]))


72.6 µs ± 309 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Heap data storage management prior to saving (dynamic vs static)

In [ ]:
def way1(n):
    x1, x2, x3 = np.tile(np.linspace(0, 2 * np.pi, n), (3,1))
    res = None
    for i in range(x1.shape[0]):
        current = np.concatenate(([x1[i]], [x2[i]], [x3[i]]))
        res = np.array([current]) if res is None else np.concatenate((res, [current]))
        
def way2(n):
    x1, x2, x3 = np.tile(np.linspace(0, 2 * np.pi, n), (3,1))
    res = np.zeros((x1.shape[0], 3))
    for i in range(x1.shape[0]):
        res[i] = np.concatenate(([x1[i]], [x2[i]], [x3[i]]))
        
results_method1 = []
for exp in [1, 2, 3, 4, 5, 6]:
    result = %timeit -o way1(10**exp)
    results_method1.append(result)
    
results_method2 = []
for exp in [1, 2, 3, 4, 5, 6]:
    result = %timeit -o way2(10**exp)
    results_method2.append(result)
    
import matplotlib.pyplot as plt
%matplotlib inline
plt.loglog([10 ** i for i in [1,2,3,4,5]], [res1.best for res1 in results_method1])
plt.loglog([10 ** i for i in [1,2,3,4,5]], [res2.best for res2 in results_method2])
plt.show()